In [0]:
import requests
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col


# API URL
api_url = "https://fakestoreapi.com/carts" 

# Fetch data from API
response = requests.get(api_url)

# Check if request was successful
if response.status_code == 200:
    data = response.json()  # Convert JSON response to Python dict
    
    # Convert list of dictionaries to RDD
    rdd = spark.sparkContext.parallelize(data)
    
    # Convert RDD to PySpark DataFrame using inferSchema
    spark_df = spark.read.json(rdd)
    df_exploded = spark_df.withColumn("product", explode(col("products")))

# # Select required fields
#     df_transformed = df_exploded.select(
#         col("id").alias("cart_id"),
#         col("userId").alias("user_id"),
#         col("date"),
#         col("product.productId").alias("product_id"),
#         col("product.quantity")
#     )

    # Show DataFrame
    df_exploded.display()
else:
    print("Failed to fetch data from API")


In [0]:
df_exploded.display()


In [0]:
# Save the DataFrame in Delta format to ADLS Gen 2
df_exploded.write.format("delta") \
    .mode("overwrite") \
    .save("abfss://bronze@nishantchurndata.dfs.core.windows.net/sales")
